In [ ]:
import requests
from pathlib import Path
import zipfile
import io

def download_patterns(target_dir: Path) -> bool:
    """Download and extract pattern files preserving directory structure."""
    url = "https://github.com/aaronksolomon/patterns/archive/main.zip"
    
    try:
        # Download zip file
        response = requests.get(url)
        response.raise_for_status()
        
        # Extract directly from memory
        with zipfile.ZipFile(io.BytesIO(response.content)) as zip_ref:
            # Files will be in a root dir like 'patterns-main/'
            root_dir = zip_ref.filelist[0].filename.split('/')[0]
            
            for zip_info in zip_ref.filelist:
                if zip_info.filename.endswith('.md'):
                    # Remove the root directory but keep nested structure
                    rel_path = Path(zip_info.filename).relative_to(root_dir)
                    target_path = target_dir / rel_path
                    
                    # Create parent directories
                    target_path.parent.mkdir(parents=True, exist_ok=True)
                    
                    # Extract file
                    with zip_ref.open(zip_info) as source, \
                         open(target_path, 'wb') as target:
                        target.write(source.read())
        return True
        
    except requests.RequestException as e:
        print(f"Download failed: {e}")
        return False
    except zipfile.BadZipFile:
        print("Invalid zip file received")
        return False

# Test it
if __name__ == "__main__":
    test_dir = Path("test_patterns")
    test_dir.mkdir(exist_ok=True)
    success = download_patterns(test_dir)
    print(f"Download {'succeeded' if success else 'failed'}")
    if success:
        print("Downloaded patterns:")
        for f in test_dir.glob("**/*.md"):
            print(f"  {f.relative_to(test_dir)}")